The yt clump finder was initially described in 
http://adsabs.harvard.edu/abs/2009ApJ...691..441S , but it's changed 
since then.  What it does now is decompose into non-overlapping tiles 
(stored in a kd-tree), identify contours within a tile, and then 
connect them across tiles.  It does this with an upper and lower bound 
on a field value, and looking for topologically connected sets. 

With yt, connected sets can be identified. This enables the location and analysis of **a hierarchy of
clumps**; in star formation, for instance, this allows the construction of diagrams **describing the density
at which fragmentation occurs**.

In [1]:
import numpy as np

import yt
from yt.analysis_modules.level_sets.api import *

ds = yt.load("output/output_00028/info_00028.txt")

yt : [INFO     ] 2018-07-03 13:40:09,567 Parameters: current_time              = 3.17276944493
yt : [INFO     ] 2018-07-03 13:40:09,568 Parameters: domain_dimensions         = [256 256 256]
yt : [INFO     ] 2018-07-03 13:40:09,569 Parameters: domain_left_edge          = [ 0.  0.  0.]
yt : [INFO     ] 2018-07-03 13:40:09,571 Parameters: domain_right_edge         = [ 1.  1.  1.]
yt : [INFO     ] 2018-07-03 13:40:09,572 Parameters: cosmological_simulation   = 1
yt : [INFO     ] 2018-07-03 13:40:09,573 Parameters: current_redshift          = 5.99989573953
yt : [INFO     ] 2018-07-03 13:40:09,574 Parameters: omega_lambda              = 0.691999971867
yt : [INFO     ] 2018-07-03 13:40:09,574 Parameters: omega_matter              = 0.307999998331
yt : [INFO     ] 2018-07-03 13:40:09,575 Parameters: hubble_constant           = 0.677900009155


In [5]:
!cat output/output_00028/*csv

In [17]:
center = [0.53103, 0.51031000000000004, 0.50402000000000002]
region_size = 0.0015
up_vector = [0.10255487134299716, 0.059509123032244614, 0.99294569974382518]

distance = 0.00075
far_cut_depth = 0.00075

data_source = ds.sphere(center, region_size)

In [4]:
ds.current_time, ds.current_redshift

(3.17276944493 code_time, 5.99989573952965)

In [9]:
data_source.get_field_parameter?

In [7]:
print(data_source(["density"]))

sp = data_source

# Print things in a more human-friendly manner: one density at a time
print("(x,  y,  z) Density")
print("-----------------------")
for i in range(sp["density"].size):
    print("(%f,  %f,  %f)    %f" %
          (sp["x"][i], sp["y"][i], sp["z"][i], sp["density"][i]))

TypeError: 'YTSphere' object is not callable

In [12]:
xx = ds.all_data()

In [13]:
xx['density']

YTArray([  1.44444555e-28,   8.29987137e-29,   1.04603599e-28, ...,
         8.99843109e-29,   1.12373516e-28,   9.79495090e-29]) g/cm**3

In [14]:
data_source.get_data?

# assuming it's disk

In [15]:
ds.disk?
# center, normal, radius, height, 

In [20]:
from yt.analysis_modules.level_sets.api import Clump, find_clumps, get_lowest_clumps
    
data_source = ds.disk(center, up_vector, region_size, (far_cut_depth + distance))

c_min = 10**np.floor(np.log10(data_source['density']).min()  )
c_max = 10**np.floor(np.log10(data_source['density']).max()+1)

master_clump = Clump(data_source, 'density')
master_clump.add_validator("min_cells", 20)

find_clumps(master_clump, c_min, c_max, 2.0)
leaf_clumps = get_lowest_clumps(master_clump)



yt : [INFO     ] 2018-07-03 13:49:36,253 Finding clumps: min: 1.000000e-28, max: 1.000000e-20, step: 2.000000


RuntimeError: AMRKDTree does not support particle or octree-based data.

In [ ]:
prj = yt.ProjectionPlot(ds, 'z', 'density', center='c', width=(20, 'kpc'))
prj.annotate_clumps(leaf_clumps)
prj.save('clumps')